## New classifiers

In [1]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

def predictpr(fitted, feature_test):
    '''
    This function predicts the probability of response 1 for the test dataset.
    
    fitted: fitted classifier
    feature_test: feature set in test data
    
    return: predictions
    '''
    return fitted.predict_proba(feature_test)[:,1]

def knn_score(x_train, y_train, x_test, n, weights = 'uniform', distance_metric = 'minkowski', p=2):
    '''
    This function builds a KNN classifier.
    
    x_train: training set with features
    y_train: training set with labels
    n: number of neighbors
    weights: weight function used in prediction.
    distance_metric: the distance metric to use
    p: Power parameter for the Minkowski metric. 
    
    returns: fitted KNN classifier
    '''
    if distance_metric == 'minkowski':
        knn = KNeighborsClassifier(n_neighbors=n, weights = weights, p=p, metric= distance_metric)
        
    else:
        knn =KNeighborsClassifier(n_neighbors=n, weights = weights, metric= distance_metric)
    
    knn.fit(x_train, y_train)
    
    return predictpr(knn, x_test)

#can use existing prediction function to predict scores

In [2]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression


def lr_score(x_train, y_train, x_test, p = 'l1', c = 1.0, solver = 'liblinear', seed=12345):
    '''
    This function builds a Logistic Regression model.
    
    x_train: training set with features
    y_train: training set with labels
    p: penalty (l1 or l2)
    c: Inverse of regularization strength; must be a positive float.
    solver: Algorithm to use in the optimization problem.
    seed: random seed
    
    returns fitted Logistic Regression
    '''
    
    lr = LogisticRegression(penalty = p, C = c, solver= solver, random_state = seed)
    lr.fit(x_train, y_train)
    
    return predictpr(lr, x_test)


#can use existing prediction function to predict scores

In [3]:
# SVM

from sklearn.svm import LinearSVC

def linsvc_score(x_train, y_train, x_test, p = 'l2', c = 1.0, seed = 12345):
    '''
    This function builds a fitted linear SVC
    
    x_train: training set with features
    y_train: training set with labels
    p: penalty (l2)
    c: Penalty parameter C of the error term
    seed: random seed
    
    returns fitted linear SVC
    '''
    
    lsvc = LinearSVC(penalty = p, C=c, random_state=seed)
    lsvc.fit(x_train, y_train)
    
    return lsvc.decision_function(x_test)

def svm_score(fitted_lsvc, x_test):
    '''
    This function uses a test set to make scores using Linear SVC.
    
    fitted_lsvc: fitted Linear SVC
    x_test: test set with features
    
    returns predictions
    '''
    return fitted_lsvc.decision_function(x_test)


In [4]:
# Random Forests

In [5]:
# Boosting

In [6]:
# Bagging

## New metrics

In [7]:
# metrics

def accuracy_at_threshold(y_test, pred_scores, thresh =0.5):
    '''
    This function calculates that accuracy of model given a threshold

    y_test: real labels
    pred: prediction scores
    threshold: threshold for predictions

    returns accuracy 
    '''
    pred_one = [1 if x > thresh else 0 for x in pred_scores]
    
    return metrics.accuracy_score(y_test, pred_one)

def build_cmatrix(y_test, pred_scores, threshold):
    '''
    This function builds a confusion matrix for a given threshold
    
    pred_scores: prediction scores
    y_test: real labels
    threshold: threshold for predictions
    
    returns tuple (true_negatives, false_positive, false_negatives, true_positives)
    '''
    pred = [1 if x > threshold else 0 for x in pred_scores]
    
    cmatrix = confusion_matrix(y_test, pred)
    
    true_negatives, false_positive, false_negatives, true_positives = cmatrix.ravel()

    return (true_negatives, false_positive, false_negatives, true_positives)

def precision_at_threshold(y_test, pred_scores, thresh =0.5):
    '''
    This function calculates precision of model given a threshold

    y_test: real labels
    pred_scores: prediction scores
    threshold: threshold for predictions

    returns precision
    '''
    pred_one = [1 if x > thresh else 0 for x in pred_scores]
    
    return metrics.precision_score(y_test, pred_one)

def recall_at_threshold(y_test, pred_scores, thresh =0.5):
    '''
    This function calculates recall of model given a threshold

    y_test: real labels
    pred_scores: prediction scores
    threshold: threshold for predictions

    returns recall
    '''
    pred_one = [1 if x > thresh else 0 for x in pred_scores]
    
    return metrics.recall_score(y_test, pred_one)

def f1_at_threshold(y_test, pred_scores, thresh =0.5):
    '''
    This function calculates that accuracy of model given a threshold

    y_test: real labels
    pred_scores: prediction scores
    threshold: threshold for predictions

    returns f1 score 
    '''
    pred_one = [1 if x > thresh else 0 for x in pred_scores]
    
    return metrics.f1_score(y_test, pred_one)

def auc_roc(y_test, pred_scores):
    '''
    This function calculates the area under the ROC curve
    
    y_test: real labels
    pred_scores: prediction scores
    
    returns auc
    '''
    
    return metrics.roc_auc_score(y_test, pred_scores)

def plot_precision_recall(y_test, pred_scores):
    '''
    This function plots the precision recall curve
    
    y_test: true labels
    pred_scores: predicted scores
    
    return: none
    '''
    precision, recall, thresholds = precision_recall_curve(y_test, pred_scores)
    plt.pyplot.plot(recall, precision, marker='.')
    plt.pyplot.show()

# HW3 Analysis

## Import and load data

In [8]:
import ml_pipeline as pp
import pandas as pd
import datetime as dt
import numpy as np
from sklearn import metrics
file = './data/projects_2012_2013.csv'
df = pp.load_csv(file)

## Data transformations

### Helper functions for hw3 specific data

#### Convert date columns to datetime

In [10]:
df.date_posted = pp.col_datetime(df, 'date_posted')

In [11]:
df.datefullyfunded = pp.col_datetime(df,'datefullyfunded')

#### Create labels

In [12]:
df = pp.create_label(df, pred_time=60)

## Selecting features and cleaning

In [13]:
feature_cols=['school_metro','school_charter', 'school_magnet', 'primary_focus_subject', 'primary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'total_price_including_optional_support', 'students_reached', 'eligible_double_your_impact_match', 'date_posted', 'label']
sel = df[feature_cols].copy()
sel.head()

,school_metro,school_charter,school_magnet,primary_focus_subject,primary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,label
0,urban,f,f,Mathematics,Math & Science,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,2013-04-14,1
1,urban,f,f,Civics & Government,History & Civics,Books,highest poverty,Grades 3-5,282.47,28.0,t,2012-04-07,1
2,urban,f,f,Literacy,Literacy & Language,Technology,high poverty,Grades 3-5,1012.38,56.0,f,2012-01-30,0
3,urban,f,t,Literacy,Literacy & Language,Books,high poverty,Grades PreK-2,175.33,23.0,f,2012-10-11,1
4,suburban,f,f,Literacy,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,2013-01-08,0


#### Identify feature columns with null values

In [14]:
for x in pp.na_col(df):
    if x in feature_cols:
        print(x)

school_metro
primary_focus_subject
primary_focus_area
resource_type
grade_level
students_reached


#### Impute missing categorical variables with the most frequent, which is a common way to handle missing categorical data without more information. 


In [15]:
cat_cols = ['school_metro','primary_focus_subject','primary_focus_area','resource_type','grade_level']
for x in cat_cols:
    sel = pp.na_fill_col(sel, x , pp.most_freq)

#### Impute missing numerical variable (students_reached) with the median value because there are outliers affecting the mean. 

In [16]:
sel.students_reached.quantile([0.1, 0.25, 0.5, 0.75, 0.9,0.98,1])

0.10       18.0
0.25       23.0
0.50       30.0
0.75      100.0
0.90      200.0
0.98      700.0
1.00    12143.0
Name: students_reached, dtype: float64

In [17]:
sel = pp.na_fill_col(sel, 'students_reached', np.nanmedian)

#### Check that there are no more missing values in feature columns

In [18]:
for x in pp.na_col(sel):
    if x in feature_cols:
        print(x)

#### Discretize numeric features and then get all dummy variables.

In [19]:
# discretize numeric features
bucketdict= {'total_price_including_optional_support': 4, 'students_reached':4}
df_discr = pp.feat_mult_disc(sel, bucketdict, qt=True)

df_discr.total_price_including_optional_support_binned.unique()
df_discr.students_reached_binned.unique()

[(30.0, 100.0], (23.0, 30.0], (0.999, 23.0], (100.0, 12143.0]]
Categories (4, interval[float64]): [(0.999, 23.0] < (23.0, 30.0] < (30.0, 100.0] < (100.0, 12143.0]]

In [20]:
col_to_binary = list(df_discr.columns)
col_to_binary.remove('label')
col_to_binary.remove('date_posted')

In [21]:
col_to_binary

['school_metro',
 'school_charter',
 'school_magnet',
 'primary_focus_subject',
 'primary_focus_area',
 'resource_type',
 'poverty_level',
 'grade_level',
 'eligible_double_your_impact_match',
 'students_reached_binned',
 'total_price_including_optional_support_binned']

In [22]:
# turn variables into dummies
df_final = pp.feat_binary(df_discr, col_to_binary)
df_final.head()

,date_posted,label,school_metro_rural,school_metro_suburban,school_metro_urban,school_charter_f,school_charter_t,school_magnet_f,school_magnet_t,primary_focus_subject_Applied Sciences,...,eligible_double_your_impact_match_f,eligible_double_your_impact_match_t,"students_reached_binned_(0.999, 23.0]","students_reached_binned_(23.0, 30.0]","students_reached_binned_(30.0, 100.0]","students_reached_binned_(100.0, 12143.0]","total_price_including_optional_support_binned_(91.999, 345.81]","total_price_including_optional_support_binned_(345.81, 510.5]","total_price_including_optional_support_binned_(510.5, 752.96]","total_price_including_optional_support_binned_(752.96, 164382.84]"
0,2013-04-14,1,0,0,1,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
1,2012-04-07,1,0,0,1,1,0,1,0,0,...,0,1,0,1,0,0,1,0,0,0
2,2012-01-30,0,0,0,1,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
3,2012-10-11,1,0,0,1,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
4,2013-01-08,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1


## Run variations of models: 
### Decision trees, KNN, Logistic Regression, Linear SVM, Random forests, Bagging, Boosting

In [38]:
windows = [dt.datetime(2012,1,1), dt.datetime(2012,7,1), dt.datetime(2013,1,1), dt.datetime(2013,7,1), dt.datetime(2014,1,1)]
pred_time = 60 #days
label_col = 'label'
split_col = 'date_posted'
feature_cols= list(df_final.columns)
feature_cols.remove('label')
feature_cols.remove('date_posted')
seed=12345

In [50]:
# models = [
#     {'type': 'Dtree', 'clf': pp.dtree_score, 'criteria': ['entropy', 'gini'], 'depth': [10,20,30],'min_leaf': [100, 300,500], 'seed': seed},
#     {'type': 'LR', 'clf': lr_score, 'p': ['l1','l2'], 'c': [0.1, 1.0, 10.0, 100.0], 'solver': ['liblinear'], 'seed': seed},
#     {'type': 'SVM', 'clf': linsvc_score, 'p': ['l2'], 'c': [0.1, 1.0, 10.0, 100.0], 'seed': seed},
#     {'type': 'KNN', 'clf': knn_score, 'n': [5,15], 'weights': ['uniform','distance'], 'distance_metric':['minkowski'], 'p': [1,2]}
# ]
models = models = [
    {'type': 'LR', 'clf': lr_score, 'p': ['l1','l2'], 'c': [0.1, 1.0, 10.0, 100.0], 'solver': ['liblinear'], 'seed': seed}]
thresholds = [0.01, 0.02, 0.05, 0.1, 0.2,0.3, 0.5]
svm_thresh = [-0.5, 0, 0.5]

In [51]:
results = []

for i in range(1, len(windows)-1):
    train_start = windows[0]
    train_end = windows[i]
    test_end = windows[i+1]
    
    #split data
    x_train,y_train,x_test,y_test = pp.single_train_test_set(df_final, 
                                                          feature_cols, 
                                                          label_col, 
                                                          split_col, 
                                                          train_start,
                                                          train_end, 
                                                          test_end, 
                                                          pred_time=pred_time)
    
    
    baseline = sum(y_test)/len(y_test)
    #run models
    for clf in models:
        modeltype = clf['type']
        func = clf['clf']
        printinfo = 'model: {}, run: {}'.format(modeltype, i)
        print(printinfo)
        if modeltype == 'Dtree':
            seed = clf['seed']
            for c in clf['criteria']:
                for d in clf['depth']:
                    for l in clf['min_leaf']:
                        #run model
                        info = 'criteria: {}, depth: {}, min_leaf: {}, seed: {}'.format(c, d, l, seed)
                        print(info)
                        scores = func(x_train, y_train, x_test, criteria = c, depth = d, min_leaf = l, seed=seed)
                        for t in thresholds:
                            acc = accuracy_at_threshold(y_test, scores, t)
                            prec = precision_at_threshold(y_test, scores, t)
                            rec = recall_at_threshold(y_test, scores, t)
                            #f1 = f1_at_threshold(y_test, scores, t)
                            auc = auc_roc(y_test,scores)
                            tmp = {'baseline': baseline, 'train_set_num': i, 'train_start': train_start, 'test_start': train_end,'type': modeltype, 
                                   'details': info, 'threshold': t, 'accuracy': acc, 'precision': prec, 'recall': rec, 'auc': auc}
                            results.append(tmp)
                            
        elif modeltype == 'LR':
            seed = clf['seed']
            for p in clf['p']:
                for c in clf['c']:
                    for s in clf['solver']:
                        #print(p)
                        info = 'penalty: {}, c: {}, solver: {}, seed: {}'.format(p, c, s, seed)
                        print(info)
                        scores = func(x_train, y_train, x_test, p = p, c = c, solver = s, seed=seed)
                        for t in thresholds:
                            acc = accuracy_at_threshold(y_test, scores, t)
                            prec = precision_at_threshold(y_test, scores, t)
                            rec = recall_at_threshold(y_test, scores, t)
                            #f1 = f1_at_threshold(y_test, scores, t)
                            auc = auc_roc(y_test, scores)
                            tmp = {'baseline': baseline,'train_set_num': i, 'train_start': train_start, 'test_start': train_end,'type': modeltype, 
                                   'details': info, 'threshold': t, 'accuracy': acc, 'precision': prec, 'recall': rec, 'auc': auc}
                            results.append(tmp)
                        
        elif modeltype == 'SVM':
            seed = clf['seed']
            for p in clf['p']:
                for c in clf['c']:
                    info = 'penalty: {}, c: {}, seed: {}'.format(p, c, seed)
                    print(info)
                    scores = func(x_train, y_train, x_test, p =p, c=c, seed=seed)
                    for t in svm_thresh:
                            acc = accuracy_at_threshold(y_test, scores, t)
                            prec = precision_at_threshold(y_test, scores, t)
                            rec = recall_at_threshold(y_test, scores, t)
                            #f1 = f1_at_threshold(y_test, scores, t)
                            auc = auc_roc(y_test, scores)
                            tmp = {'baseline': baseline,'train_set_num': i, 'train_start': train_start, 'test_start': train_end,'type': modeltype, 
                                   'details': info, 'threshold': t, 'accuracy': acc, 'precision': prec, 'recall': rec, 'auc': auc}
                            results.append(tmp)
                            
        elif modeltype == 'KNN':
            for n in clf['n']:
                for w in clf['weights']:
                    for d in clf['distance_metric']:
                        for p in clf['p']:
                            info = 'n: {}, weights: {}, distance: {}, p: {}'.format(n, w, d, p)
                            print(info)
                            scores = func(x_train, y_train, x_test, n = n, weights = w, distance_metric = d, p=p)
                            #print(list(scores))
                            for t in thresholds:
                                acc = accuracy_at_threshold(y_test, scores, t)
                                prec = precision_at_threshold(y_test, scores, t)
                                rec = recall_at_threshold(y_test, scores, t)
                                #f1 = f1_at_threshold(y_test, scores, t)
                                auc = auc_roc(y_test,scores)
                                tmp = {'baseline': baseline,'train_set_num': i, 'train_start': train_start, 'test_start': train_end,'type': modeltype, 
                                       'details': info, 'threshold': t, 'accuracy': acc, 'precision': prec, 'recall': rec, 'auc': auc}
                                results.append(tmp)
                    
resdf=pd.DataFrame(results, columns = ['type', 'details', 'baseline', 'threshold', 'accuracy', 'precision', 'recall', 'auc','train_set_num', 'train_start', 'test_start'])    

model: LR, run: 1
penalty: l1, c: 0.1, solver: liblinear, seed: 12345
penalty: l1, c: 1.0, solver: liblinear, seed: 12345
penalty: l1, c: 10.0, solver: liblinear, seed: 12345
penalty: l1, c: 100.0, solver: liblinear, seed: 12345
penalty: l2, c: 0.1, solver: liblinear, seed: 12345
penalty: l2, c: 1.0, solver: liblinear, seed: 12345
penalty: l2, c: 10.0, solver: liblinear, seed: 12345
penalty: l2, c: 100.0, solver: liblinear, seed: 12345
model: LR, run: 2
penalty: l1, c: 0.1, solver: liblinear, seed: 12345
penalty: l1, c: 1.0, solver: liblinear, seed: 12345
penalty: l1, c: 10.0, solver: liblinear, seed: 12345
penalty: l1, c: 100.0, solver: liblinear, seed: 12345
penalty: l2, c: 0.1, solver: liblinear, seed: 12345
penalty: l2, c: 1.0, solver: liblinear, seed: 12345
penalty: l2, c: 10.0, solver: liblinear, seed: 12345
penalty: l2, c: 100.0, solver: liblinear, seed: 12345
model: LR, run: 3
penalty: l1, c: 0.1, solver: liblinear, seed: 12345
penalty: l1, c: 1.0, solver: liblinear, seed: 1234

In [52]:
resdf[resdf['type']=='LR']

,type,details,baseline,threshold,accuracy,precision,recall,auc,train_set_num,train_start,test_start
0,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.743083,0.01,0.743083,0.743083,1.000000,0.662580,1,2012-01-01,2012-07-01
1,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.743083,0.02,0.743083,0.743083,1.000000,0.662580,1,2012-01-01,2012-07-01
2,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.743083,0.05,0.743083,0.743083,1.000000,0.662580,1,2012-01-01,2012-07-01
3,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.743083,0.10,0.743083,0.743083,1.000000,0.662580,1,2012-01-01,2012-07-01
4,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.743083,0.20,0.743083,0.743083,1.000000,0.662580,1,2012-01-01,2012-07-01
5,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.743083,0.30,0.743083,0.743083,1.000000,0.662580,1,2012-01-01,2012-07-01
6,LR,"penalty: l1, c: 0.1, solver: liblinear, seed: ...",0.743083,0.50,0.731098,0.760770,0.930835,0.662580,1,2012-01-01,2012-07-01
7,LR,"penalty: l1, c: 1.0, solver: liblinear, seed: ...",0.743083,0.01,0.743083,0.743083,1.000000,0.663154,1,2012-01-01,2012-07-01
8,LR,"penalty: l1, c: 1.0, solver: liblinear, seed: ...",0.743083,0.02,0.743083,0.743083,1.000000,0.663154,1,2012-01-01,2012-07-01
9,LR,"penalty: l1, c: 1.0, solver: liblinear, seed: ...",0.743083,0.05,0.743083,0.743083,1.000000,0.663154,1,2012-01-01,2012-07-01


In [37]:
resdf.to_csv('firstrun.csv')

,type,details,baseline,threshold,accuracy,precision,recall,auc,train_set_num,train_start,test_start
0,Dtree,"criteria: entropy, depth: 5, min_leaf: 300, se...",0.743083,0.10,0.743083,0.743083,1.000000,0.647525,1,2012-01-01,2012-07-01
1,Dtree,"criteria: entropy, depth: 5, min_leaf: 300, se...",0.743083,0.20,0.743083,0.743083,1.000000,0.647525,1,2012-01-01,2012-07-01
2,Dtree,"criteria: entropy, depth: 5, min_leaf: 300, se...",0.743083,0.05,0.743083,0.743083,1.000000,0.647525,1,2012-01-01,2012-07-01
3,Dtree,"criteria: entropy, depth: 5, min_leaf: 300, se...",0.743083,0.10,0.743083,0.743083,1.000000,0.647525,1,2012-01-01,2012-07-01
4,Dtree,"criteria: entropy, depth: 5, min_leaf: 300, se...",0.743083,0.20,0.743083,0.743083,1.000000,0.647525,1,2012-01-01,2012-07-01
5,Dtree,"criteria: entropy, depth: 5, min_leaf: 300, se...",0.743083,0.30,0.743083,0.743083,1.000000,0.647525,1,2012-01-01,2012-07-01
6,Dtree,"criteria: entropy, depth: 5, min_leaf: 300, se...",0.743083,0.50,0.726972,0.758795,0.927364,0.647525,1,2012-01-01,2012-07-01
7,Dtree,"criteria: entropy, depth: 5, min_leaf: 500, se...",0.743083,0.10,0.743083,0.743083,1.000000,0.647834,1,2012-01-01,2012-07-01
8,Dtree,"criteria: entropy, depth: 5, min_leaf: 500, se...",0.743083,0.20,0.743083,0.743083,1.000000,0.647834,1,2012-01-01,2012-07-01
9,Dtree,"criteria: entropy, depth: 5, min_leaf: 500, se...",0.743083,0.05,0.743083,0.743083,1.000000,0.647834,1,2012-01-01,2012-07-01


In [27]:
#### Use temporal validation to split into training/test sets



x_train1,y_train1,x_test1,y_test1 = pp.single_train_test_set(df_final, 
                                                          feature_cols, 
                                                          label_col, 
                                                          split_col, 
                                                          windows[0],
                                                          windows[1], 
                                                          windows[2], 
                                                          pred_time=60)

x_train2, y_train2, x_test2, y_test2 = pp.single_train_test_set(df_final, 
                                                          feature_cols, 
                                                          label_col, 
                                                          split_col, 
                                                          windows[0],
                                                          windows[2], 
                                                          windows[3], 
                                                          pred_time=60)

x_train3, y_train3, x_test3, y_test3 = pp.single_train_test_set(df_final, 
                                                          feature_cols, 
                                                          label_col, 
                                                          split_col, 
                                                          windows[0],
                                                          windows[3], 
                                                          windows[4], 
                                                          pred_time=60)

In [31]:
sum(y_test1)

24492

In [74]:
clf = knn_score(x_train1, y_train1, x_test1, n=3, p=1)
clf

array([0.66666667, 0.66666667, 1.        , ..., 1.        , 1.        ,
       0.66666667])

In [75]:
accuracy_at_threshold(y_test1, clf, 0.5)

0.6534890776699029

In [76]:
list(clf)

[0.6666666666666666,
 0.6666666666666666,
 1.0,
 0.6666666666666666,
 0.0,
 0.3333333333333333,
 1.0,
 1.0,
 0.0,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 1.0,
 0.6666666666666666,
 0.3333333333333333,
 1.0,
 0.6666666666666666,
 0.3333333333333333,
 0.6666666666666666,
 0.3333333333333333,
 0.0,
 0.3333333333333333,
 0.6666666666666666,
 0.6666666666666666,
 0.3333333333333333,
 1.0,
 0.6666666666666666,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.3333333333333333,
 1.0,
 1.0,
 0.3333333333333333,
 1.0,
 1.0,
 1.0,
 1.0,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 1.0,
 0.6666666666666666,
 0.6666666666666666,
 1.0,
 1.0,
 0.6666666666666666,
 0.3333333333333333,
 1.0,
 1.0,
 0.6666666666666666,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 1.0,
 0.6666666666666666,
 0.3333333333333333,
 0.6666666666666666,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 1.0,
 1.0,
 1.0,
 0.3333333333333333,
 0.3333

In [53]:
x=lr_score(x_train1, y_train1, x_test1, p = 'l1', c = 0.1, solver = 'liblinear', seed=12345)

In [59]:
recall_at_threshold(y_test1, x, 0.2)

1.0

In [60]:
p = [1 if i > 0.3 else 0 for i in x]

In [62]:
metrics.recall_score(y_test1, p)

1.0

In [65]:
np.min(x)

0.37315420936474564

In [64]:
list(p)

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [63]:
list(y_test1)

[1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
